In [0]:
gold_df = spark.read.parquet("/Volumes/firstproject1/default/silver/superstore/")

In [0]:
from pyspark.sql.functions import regexp_replace, col

gold_df = gold_df.withColumn(
    "Sales",
    regexp_replace(col("Sales"), "[^0-9.]", "")
)

gold_df = gold_df.withColumn(
    "Sales",
    col("Sales").cast("double")
)


In [0]:
from pyspark.sql.functions import col, when, regexp_replace, expr

gold_df = gold_df.withColumn(
    "Sales",
    when(col("Sales") == "", None).otherwise(col("Sales"))
)

gold_df = gold_df.withColumn(
    "Sales",
    regexp_replace(col("Sales"), "[^0-9.]", "")
)

gold_df = gold_df.withColumn(
    "Sales",
    expr("try_cast(Sales as double)")
)


In [0]:
from pyspark.sql.functions import*


**Monthly Sales Trend**

In [0]:
monthly_sales = gold_df.groupBy("Order Year", "Order Month").agg(sum("Sales").alias("monthly_sales"))




**Profit by Region**

In [0]:
profit_by_region = gold_df.groupBy("Region").agg(sum("Profit").alias("total_profit"))


In [0]:
profit_by_region.write \
    .mode("overwrite") \
    .parquet("/Volumes/firstproject1/default/gold/profit_by_region")


**Loss Orders Summary**

In [0]:
loss_summary = gold_df.groupBy("Loss Flag").agg(sum("Sales").alias("total_sales"),sum("Profit").alias("total_profit"))


**Top 10 Customers by Sales**

In [0]:
top_customers = gold_df.groupBy("Customer ID", "Customer Name") \
    .agg(
        sum("Sales").alias("total_sales")
    ) \
    .orderBy(desc("total_sales")).limit(10)


**Discount Band**

In [0]:
df_discount = gold_df.withColumn(
    "Discount Band",
    when(gold_df["Discount"] == 0, "No Discount")
    .when(gold_df["Discount"] <= 0.2, "Low")
    .when(gold_df["Discount"] <= 0.5, "Medium")
    .otherwise("High")
)


In [0]:
clv = gold_df.groupBy("Customer ID", "Customer Name") \
    .agg(
        sum("Sales").alias("lifetime_sales"),
        sum("Profit").alias("lifetime_profit")
    )
